In [1]:
import orjson

from conda_oci_utils import (
    encode_conda_dist_to_oci_dist,
    is_valid_oci_dist,
    is_valid_conda_dist,
)
from conda_forge_metadata.repodata import fetch_repodata, all_labels

In [2]:
import tqdm

labels = all_labels(use_remote_cache=True)

max_name_len = 0
max_tag_len = 0
for label in tqdm.tqdm(labels, ncols=80):
    for subdir in [
        "linux-64",
        "osx-64",
        "noarch",
        "win-64",
        "linux-aarch64",
        "osx-arm64",
        "linux-ppc64le",
    ]:
        rd = fetch_repodata([subdir], label=label)[0]
        with open(rd) as f:
            rd = orjson.loads(f.read())

        for key in ["packages", "packages.conda"]:
            for dist in rd[key]:
                if not is_valid_conda_dist(dist):
                    tqdm.tqdm.write(f"invalid conda dist: {dist} (label {label})")

                if label != "main":
                    conda_dist = f"conda-forge/{label}/{subdir}/{dist}"
                else:
                    conda_dist = f"conda-forge/{subdir}/{dist}"

                if not is_valid_conda_dist(conda_dist):
                    tqdm.tqdm.write(f"invalid conda dist: {conda_dist} (label: {label})")

                oci_dist = encode_conda_dist_to_oci_dist(conda_dist)

                if not is_valid_oci_dist(oci_dist):
                    tqdm.tqdm.write(f"invalid oci dist: {oci_dist} (label: {label})")
                else:
                    name, tag = oci_dist.rsplit(":", maxsplit=1)
                    # if len(name) > max_name_len:
                    #     tqdm.tqdm.write(f"current maximum oci name length: {len(name)}")
                    max_name_len = max(max_name_len, len(name))
                    # if len(tag) > max_tag_len:
                    #     tqdm.tqdm.write(f"current maximum oci tag length: {len(tag)}")
                    max_tag_len = max(max_tag_len, len(tag))
                    # tqdm.tqdm.write(f"valid oci dist: {oci_dist} (label: {label})")

print(f"max oci name length: {max_name_len}")
print(f"max oci tag length: {max_tag_len}")

  7%|██▊                                      | 25/358 [00:00<00:02, 124.44it/s]

invalid conda dist: pyside2-2.0.0~alpha0-py27_0.tar.bz2 (label broken-and-invalid)
invalid conda dist: conda-forge/broken-and-invalid/linux-64/pyside2-2.0.0~alpha0-py27_0.tar.bz2 (label: broken-and-invalid)
invalid oci dist: conda-forge/linux-64/cpyside2:2.0.0~alpha0-py27_U0-broken_Dand_Dinvalid (label: broken-and-invalid)
invalid conda dist: pyside2-2.0.0~alpha0-py35_0.tar.bz2 (label broken-and-invalid)
invalid conda dist: conda-forge/broken-and-invalid/linux-64/pyside2-2.0.0~alpha0-py35_0.tar.bz2 (label: broken-and-invalid)
invalid oci dist: conda-forge/linux-64/cpyside2:2.0.0~alpha0-py35_U0-broken_Dand_Dinvalid (label: broken-and-invalid)
invalid conda dist: pyside2-2.0.0~alpha0-py36_0.tar.bz2 (label broken-and-invalid)
invalid conda dist: conda-forge/broken-and-invalid/linux-64/pyside2-2.0.0~alpha0-py36_0.tar.bz2 (label: broken-and-invalid)
invalid oci dist: conda-forge/linux-64/cpyside2:2.0.0~alpha0-py36_U0-broken_Dand_Dinvalid (label: broken-and-invalid)
invalid conda dist: pysid

100%|█████████████████████████████████████████| 358/358 [00:34<00:00, 10.25it/s]

max oci name length: 71
max oci tag length: 88


In [4]:
import defaults_repodata

for channel in defaults_repodata.DEFAULTS_CHANNELS:
    labels = ["main", "borken", "borked"]
    # does not work right now
    # labels = defaults_repodata.all_labels(channel=channel)
    # print(channel, labels)

    max_name_len = 0
    max_tag_len = 0
    for label in tqdm.tqdm(labels, ncols=80, desc=channel):
        for subdir in [
            "linux-64",
            "osx-64",
            "noarch",
            "win-64",
            "linux-aarch64",
            "osx-arm64",
            "linux-ppc64le",
        ]:
            rd = defaults_repodata.fetch_repodata(channel, [subdir], label=label)[0]
            with open(rd) as f:
                rd = orjson.loads(f.read())

            for key in ["packages", "packages.conda"]:
                for dist in rd[key]:
                    if (
                        (not is_valid_conda_dist(dist))
                        and "__anaconda_core_depends" not in dist
                    ):
                        tqdm.tqdm.write(f"invalid conda dist: {dist} (label {label})")

                    if label != "main":
                        conda_dist = f"{channel}/{label}/{subdir}/{dist}"
                    else:
                        conda_dist = f"{channel}/{subdir}/{dist}"

                    if (
                        (not is_valid_conda_dist(conda_dist))
                        and "__anaconda_core_depends" not in conda_dist
                    ):
                        tqdm.tqdm.write(f"invalid conda dist: {conda_dist} (label: {label})")

                    oci_dist = encode_conda_dist_to_oci_dist(conda_dist)

                    if not is_valid_oci_dist(oci_dist):
                        tqdm.tqdm.write(f"invalid oci dist: {oci_dist} (label: {label})")
                    else:
                        name, tag = oci_dist.rsplit(":", maxsplit=1)
                        # if len(name) > max_name_len:
                        #     tqdm.tqdm.write(f"current maximum oci name length: {len(name)}")
                        max_name_len = max(max_name_len, len(name))
                        # if len(tag) > max_tag_len:
                        #     tqdm.tqdm.write(f"current maximum oci tag length: {len(tag)}")
                        max_tag_len = max(max_tag_len, len(tag))
                        # tqdm.tqdm.write(f"valid oci dist: {oci_dist} (label: {label})")

    print(f"max oci name length: {max_name_len}")
    print(f"max oci tag length: {max_tag_len}")

main: 100%|███████████████████████████████████████| 3/3 [00:01<00:00,  1.57it/s]


max oci name length: 64
max oci tag length: 48


msys2: 100%|██████████████████████████████████████| 3/3 [00:01<00:00,  1.91it/s]


max oci name length: 44
max oci tag length: 21


r: 100%|██████████████████████████████████████████| 3/3 [00:07<00:00,  2.62s/it]

max oci name length: 41
max oci tag length: 30
